# The Task

Find a risky transaction that will become a chargeback (state=chargeback or refund+fees)

Enclosed are 
- Transactions with their meta data
- Subscriptions, Each transaction create a subscription object
- User attributes, what we collected about a user

User identified by ClientID and by a GUID (a PC unique stamp, usually 1 pc = 1 user)

There is a lot of extra info that may or may not be used

Transactions
https://fast-test-datasci.s3.amazonaws.com/trans.rar

Subscriptions
https://fast-test-datasci.s3.amazonaws.com/ccsubs.rar

User Attribute
https://fast-test-datasci.s3.amazonaws.com/attr.rar


## >Використані бібліотеки

In [2]:
from sklearn import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## >Завантаження даних 

In [3]:
df1 = pd.read_csv('Атрибут користувача 1.csv')

In [5]:
df2 = pd.read_csv('Атрибут користувача 2.csv')

C:\Users\BU400A\AppData\Local\Temp\ipykernel_9628\3186476912.py:1: DtypeWarning: Columns (52) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('Атрибут користувача 2.csv')


In [7]:
df3 = pd.read_csv('Атрибут користувача 3.csv')

In [6]:
df4 = pd.read_csv('Атрибут користувача 4.csv')

C:\Users\BU400A\AppData\Local\Temp\ipykernel_9628\4185521652.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df4 = pd.read_csv('Атрибут користувача 4.csv')


In [8]:
df5 = pd.read_csv('Атрибут користувача 5.csv')

In [9]:
transac = pd.read_csv('Transactions.txt', delimiter='\t')

C:\Users\BU400A\AppData\Local\Temp\ipykernel_9628\2270557742.py:1: DtypeWarning: Columns (0,1,2,6,7,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  transac = pd.read_csv('Transactions.txt', delimiter='\t')


In [10]:
ccsubs = pd.read_csv('CC and subscriptions.txt', delimiter='\t')

## >Виведення основної інформації про дані

In [11]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3747 entries, 0 to 3746
Columns: 148 entries, id to d
dtypes: float64(15), int64(70), object(63)
memory usage: 4.2+ MB


In [12]:
df1.head()

,id,referrer,src_domain,src_url,campaign_last,campaign_first,at,ai,same_campaign,campaign_visits,...,user_agent,ip,geo,ip_city,ip_state_iso,ip_state_name,ip_tz,ip_tzo,risk,d
0,4568921,-,NaN,NaN,activeTrail,activeTrail,NaN,NaN,0,0,...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15) ...,24.140.168.18,US,Massillon,OH,Ohio,America/New_York,240,{},200210
1,4569179,-,NaN,NaN,activeTrail,activeTrail,NaN,NaN,0,0,...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,92.5.101.53,GB,Doncaster,DNC,Doncaster,Europe/London,-60,{},200210
2,4569217,-,NaN,NaN,activeTrail,activeTrail,NaN,NaN,0,0,...,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3_1 like...,86.162.82.27,GB,Morpeth,NBL,Northumberland,Europe/London,-60,{},200210
3,4337476,-,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3_1 like...,73.53.100.16,US,Auburn,WA,Washington,America/Los_Angeles,420,{},200210
4,4569451,-,NaN,NaN,activeTrail,activeTrail,NaN,NaN,0,0,...,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3_1 like...,73.206.136.220,US,Houston,TX,Texas,America/Chicago,300,{},200210


In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4659 entries, 0 to 4658
Columns: 148 entries, id to d
dtypes: float64(15), int64(71), object(62)
memory usage: 5.3+ MB


In [17]:
df2.head()

,id,referrer,src_domain,src_url,campaign_last,campaign_first,at,ai,same_campaign,campaign_visits,...,user_agent,ip,geo,ip_city,ip_state_iso,ip_state_name,ip_tz,ip_tzo,risk,d
0,11896214,https://www.bing.com/,bing.com,www.bing.com/,topav_au,topav_au,NaN,NaN,1,1,...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,110.140.32.76,AU,Melbourne,VIC,Victoria,Australia/Melbourne,-600,{},210527
1,12079640,https://googleads.g.doubleclick.net/,googleads.g.doubleclick.net,googleads.g.doubleclick.net/,wdl_fa_ww,wdl_fa_ww,NaN,software,1,1,...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,118.210.63.201,AU,Blackwood,SA,South Australia,Australia/Adelaide,-15450,{},200610
2,12115595,https://googleads.g.doubleclick.net/,googleads.g.doubleclick.net,googleads.g.doubleclick.net/,wdl_fa_ww_mcf,wdl_fa_ww_mcf,NaN,software,1,1,...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,174.219.3.189,US,Saint Paul,MN,Minnesota,America/Chicago,300,{},210827
3,4676792,-,NaN,NaN,activeTrail,em_msoffice_1_1,NaN,opt10000d,1,1,...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,66.249.84.184,US,Spruce Grove,AB,Alberta,America/Edmonton,780,{},200417
4,11949247,https://googleads.g.doubleclick.net/pagead/ads...,popsilla.com,www.popsilla.com/au.com.kayosports,wdl_2x2min,wdl_2x2min,NaN,software,1,2,...,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,49.199.16.62,AU,Richmond,VIC,Victoria,Australia/Melbourne,-600,{},210620


In [14]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4055 entries, 0 to 4054
Columns: 148 entries, id to d
dtypes: float64(14), int64(71), object(63)
memory usage: 4.6+ MB


In [18]:
df3.head()

,id,referrer,src_domain,src_url,campaign_last,campaign_first,at,ai,same_campaign,campaign_visits,...,user_agent,ip,geo,ip_city,ip_state_iso,ip_state_name,ip_tz,ip_tzo,risk,d
0,28977539,https://www.google.com/,google.com,www.google.com/,telegram_search,telegram_search,NaN,NaN,1,1,...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,102.165.48.100,US,Washington,DC,District of Columbia,America/New_York,240,"{""mac"": {""gateway"": ""C8EAF849F00E"", ""net_local...",221012
1,10185115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,141.226.140.100,US,Copenhagen,84,Capital Region,Europe/Copenhagen,-120,{},201127
2,11065006,https://googleads.g.doubleclick.net/,googleads.g.doubleclick.net,googleads.g.doubleclick.net/,wdl_fa_ww3,wdl_fa_ww3,businessProductivitySoftware,mobileEnthusiasts,1,1,...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,99.228.247.235,CA,Maple,ON,Ontario,America/Toronto,540,{},201227
3,11088770,https://googleads.g.doubleclick.net/,googleads.g.doubleclick.net,googleads.g.doubleclick.net/,sms,zzy,NaN,NaN,0,4,...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,1.157.9.60,AU,Gungahlin,ACT,Australian Capital Territory,Australia/Sydney,-600,{},210106
4,11090271,https://googleads.g.doubleclick.net/,googleads.g.doubleclick.net,googleads.g.doubleclick.net/,wdy,wdy,businessProductivitySoftware,software,1,1,...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,99.6.232.251,US,Dallas,TX,Texas,America/Chicago,300,{},210106


In [15]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5116 entries, 0 to 5115
Columns: 148 entries, id to d
dtypes: float64(17), int64(71), object(60)
memory usage: 5.8+ MB


In [19]:
df4.head()

,id,referrer,src_domain,src_url,campaign_last,campaign_first,at,ai,same_campaign,campaign_visits,...,user_agent,ip,geo,ip_city,ip_state_iso,ip_state_name,ip_tz,ip_tzo,risk,d
0,26642248,https://googleads.g.doubleclick.net/,googleads.g.doubleclick.net,googleads.g.doubleclick.net/,wdl_fa_ww4,wdl_fa_ww4,NaN,NaN,1,1,...,Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537....,99.247.34.202,CA,Mississauga,ON,Ontario,America/Toronto,300,"{""ip"": {""iprisk"": ""0.01"", ""riskscore"": ""0.17""}...",220222
1,26642423,https://googleads.g.doubleclick.net/,googleads.g.doubleclick.net,googleads.g.doubleclick.net/,wdl_fa_legacy,wdl_fa_legacy,NaN,mcafeeOffice,1,1,...,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,184.144.155.15,CA,Salaberry-de-Valleyfield,QC,Quebec,America/Toronto,300,"{""mac"": {""gateway"": ""C03C041D5768"", ""net_local...",220222
2,26642479,https://googleads.g.doubleclick.net/,googleads.g.doubleclick.net,googleads.g.doubleclick.net/,wdl_fa_legacy,wdl_fa_legacy,NaN,mcafeeOffice,1,1,...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,1.158.56.45,AU,Annandale,NSW,New South Wales,Australia/Sydney,-660,"{""ip"": {""iprisk"": ""0.01"", ""riskscore"": ""0.1""},...",220222
3,26642817,https://googleads.g.doubleclick.net/,googleads.g.doubleclick.net,googleads.g.doubleclick.net/,wdl_fa_ww4,wdl_fa_ww4,androidApps,HostedDataCloudStorage,1,1,...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,108.180.15.39,CA,Campbell River,BC,British Columbia,America/Vancouver,900,"{""ip"": {""iprisk"": ""0.01"", ""riskscore"": ""0.1""},...",220222
4,26642837,https://googleads.g.doubleclick.net/,googleads.g.doubleclick.net,googleads.g.doubleclick.net/,wdl_fa_legacy,wdl_fa_legacy,NaN,mcafeeOffice,1,2,...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,58.168.208.94,AU,Chatswood,NSW,New South Wales,Australia/Sydney,-660,"{""ip"": {""iprisk"": ""0.01"", ""riskscore"": ""0.1""},...",220222


In [16]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3848 entries, 0 to 3847
Columns: 148 entries, id to d
dtypes: float64(16), int64(70), object(62)
memory usage: 4.3+ MB


In [20]:
df5.head()

,id,referrer,src_domain,src_url,campaign_last,campaign_first,at,ai,same_campaign,campaign_visits,...,user_agent,ip,geo,ip_city,ip_state_iso,ip_state_name,ip_tz,ip_tzo,risk,d
0,4928742,-,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,NaN,69.147.90.254,US,Lake Oswego,OR,Oregon,America/Los_Angeles,420,{},200516
1,7812729,-,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,Mozilla/5.0 (iPhone; CPU iPhone OS 13_4_1 like...,184.148.36.66,CA,St. Catharines,ON,Ontario,America/Toronto,240,{},200516
2,9757139,https://googleads.g.doubleclick.net/pagead/ads...,appdirectory.online,www.appdirectory.online/download/itunes/,wdl_fa_ww,wdl_fa_ww,NaN,software,1,1,...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,134.41.223.52,CA,Bay Roberts,NL,Newfoundland and Labrador,America/St_Johns,150,{},200516
3,4804078,-,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,70.120.213.17,US,Uvalde,TX,Texas,America/Chicago,300,{},200516
4,4802995,-,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,69.147.90.62,US,Mountain Ranch,CA,California,America/Los_Angeles,720,{},200518


In [21]:
transac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92167 entries, 0 to 92166
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               92167 non-null  object 
 1   customer_id      92167 non-null  object 
 2   subscription_id  92167 non-null  object 
 3   guid             91609 non-null  object 
 4   email            90421 non-null  object 
 5   geo              91695 non-null  object 
 6   amount           92166 non-null  object 
 7   returns          92166 non-null  object 
 8   item_name        91927 non-null  object 
 9   item_key         80821 non-null  object 
 10  campaign         76931 non-null  object 
 11  state            92120 non-null  object 
 12  created_time     92165 non-null  float64
 13  provider         92161 non-null  object 
 14  oid              39268 non-null  object 
 15  descriptor       65556 non-null  object 
 16  strategy_id      40839 non-null  object 
 17  strategy_con

In [22]:
transac.head()

,id,customer_id,subscription_id,guid,email,geo,amount,returns,item_name,item_key,campaign,state,created_time,provider,oid,descriptor,strategy_id,strategy_config,json
0,11,7,0,03000200-0400-0500-0006-000700080009,email@btinternet.com,IL,2.95,0,Fast! (monthly),NaN,batch001,Completed,1.489680e+09,UpClick,NaN,NaN,NaN,{},"{""id"": ""5"", ""chk"": ""1FE690276694C82917A14E5CD1..."
1,13,9,0,B9B14F7F-76FC-11DD-B661-DDA0BF0C50C1,email@eletewireless.co.za,ZA,2.95,0,Fast! (monthly),NaN,NaN,Completed,1.489919e+09,UpClick,NaN,NaN,NaN,{},"{""id"": ""7"", ""chk"": ""D170AA7E27F35C557F7B1298CC..."
2,14,10,0,2A183F20-3890-11DF-9257-7071BC05CE48,email@iafrica.com,ZA,49.95,0,Fast! (annual),NaN,NaN,Completed,1.490161e+09,UpClick,NaN,NaN,NaN,{},"{""id"": ""8"", ""chk"": ""EBA378D55BE333A21804A32ABF..."
3,15,11,0,30464E43-3035-5838-3636-78ACC049073C,email@hotmail.com,TR,2.95,0,Fast! (monthly),NaN,NaN,Completed,1.490196e+09,UpClick,NaN,NaN,NaN,{},"{""id"": ""9"", ""chk"": ""EC1F5BE747C6562C1B4CCCD690..."
4,16,12,0,515F4572-DBEC-CB41-A8CD-2EF2BEAB71DE,email@gmail.com,MY,2.95,0,Fast! (monthly),NaN,NaN,Completed,1.490203e+09,UpClick,NaN,NaN,NaN,{},"{""id"": ""10"", ""chk"": ""DA8E21E922F97C1F5D5E259E7..."


In [23]:
ccsubs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60932 entries, 0 to 60931
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 60932 non-null  int64  
 1   customer_id        60932 non-null  int64  
 2   guid               60931 non-null  object 
 3   page               56994 non-null  object 
 4   shopper_id         48485 non-null  float64
 5   created_time       60932 non-null  int64  
 6   last_billing_time  60932 non-null  int64  
 7   next_billing_time  60932 non-null  int64  
 8   amount             60932 non-null  float64
 9   ltv                60932 non-null  float64
 10  interval_sec       60932 non-null  int64  
 11  state              60932 non-null  object 
 12  retry_count        60932 non-null  int64  
 13  descriptor         54993 non-null  object 
 14  meta_data          60932 non-null  object 
 15  log_json           60932 non-null  object 
 16  strategy_id        370

In [24]:
ccsubs.head()

,id,customer_id,guid,page,shopper_id,created_time,last_billing_time,next_billing_time,amount,ltv,interval_sec,state,retry_count,descriptor,meta_data,log_json,strategy_id,strategy_config
0,1,44444,FAB0479A-96FD-4F73-9FBE-B418C9F47B69,NaN,116923660.0,1601072965,1603665060,1603837860,9.99,139.86,2592000,declined,1,NaN,{},"[\n {\n ""time"": 1567028797,\n ...",NaN,{}
1,3,44493,1DC99082-1185-11DF-BBDA-85804582D8D3,NaN,117013344.0,1577488083,1577488083,1580080083,9.99,49.95,2592000,canceled,0,NaN,{},"[{""time"":1567119645,""comment"":""success purchas...",NaN,{}
2,4,44510,35444335-3334-4B35-3956-B05ADADD0E80,NaN,117180938.0,1596786634,1596786634,1596786387,9.99,109.89,2592000,failed,5,NaN,{},"[\n {\n ""time"": 1567187352,\n ...",NaN,{}
3,5,44513,4C4C4544-0030-3910-8044-B9C04F545231,NaN,118633050.0,1574982120,1574982120,0,9.99,29.97,2592000,failed,12,NaN,{},"[{""time"":1568255087,""comment"":""success purchas...",NaN,{}
4,6,44599,4C4C4544-004E-4710-8034-C6C04F475432,NaN,117852486.0,1572824502,1572824502,0,4.99,14.97,2592000,failed,10,NaN,{},"[{""time"":1567640136,""comment"":""success purchas...",NaN,{}


## >Попередня обробка даних

Оскільки набори даних df1, df2, df3, df4, df5 мають однакову кількість колонок і повністю співпадають за їх назвами, тому є сенс об'єднати дані набори в один

In [25]:
df = pd.concat([df1, df2, df3, df4, df5])

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21425 entries, 0 to 3847
Columns: 148 entries, id to d
dtypes: float64(15), int64(70), object(63)
memory usage: 24.4+ MB


In [28]:
df.head()

,id,referrer,src_domain,src_url,campaign_last,campaign_first,at,ai,same_campaign,campaign_visits,...,user_agent,ip,geo,ip_city,ip_state_iso,ip_state_name,ip_tz,ip_tzo,risk,d
0,4568921,-,NaN,NaN,activeTrail,activeTrail,NaN,NaN,0,0,...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15) ...,24.140.168.18,US,Massillon,OH,Ohio,America/New_York,240,{},200210
1,4569179,-,NaN,NaN,activeTrail,activeTrail,NaN,NaN,0,0,...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,92.5.101.53,GB,Doncaster,DNC,Doncaster,Europe/London,-60,{},200210
2,4569217,-,NaN,NaN,activeTrail,activeTrail,NaN,NaN,0,0,...,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3_1 like...,86.162.82.27,GB,Morpeth,NBL,Northumberland,Europe/London,-60,{},200210
3,4337476,-,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3_1 like...,73.53.100.16,US,Auburn,WA,Washington,America/Los_Angeles,420,{},200210
4,4569451,-,NaN,NaN,activeTrail,activeTrail,NaN,NaN,0,0,...,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3_1 like...,73.206.136.220,US,Houston,TX,Texas,America/Chicago,300,{},200210


In [ ]:
# Ось тут умене питання чи об'днувати інші 2 таблиці у цей набор даних, чи окремо їх розглядати????

In [30]:
df_all = pd.concat([df, transac, ccsubs])

In [31]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174524 entries, 0 to 60931
Columns: 172 entries, id to log_json
dtypes: float64(90), object(82)
memory usage: 230.4+ MB


In [32]:
df_all.head()

,id,referrer,src_domain,src_url,campaign_last,campaign_first,at,ai,same_campaign,campaign_visits,...,strategy_config,json,page,shopper_id,last_billing_time,next_billing_time,interval_sec,retry_count,meta_data,log_json
0,4568921,-,NaN,NaN,activeTrail,activeTrail,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4569179,-,NaN,NaN,activeTrail,activeTrail,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4569217,-,NaN,NaN,activeTrail,activeTrail,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4337476,-,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4569451,-,NaN,NaN,activeTrail,activeTrail,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df_all.isna().sum().div(df_all.shape[0]).mul(100).round(2)

id                    0.00
referrer             87.75
src_domain           89.00
src_url              89.00
campaign_last        88.23
                     ...  
next_billing_time    65.09
interval_sec         65.09
retry_count          65.09
meta_data            65.09
log_json             65.09
Length: 172, dtype: float64

In [ ]:
# Я розумію, що нема сенсу все об'єднувати, оскільки в наборі даних з'являється величезна кількість пропущених значень

#### Виведення пропущених значень у відсотках  по усім колонкам (одночасно бачу назви всіх колонок)

In [55]:
df.isna().sum().div(df.shape[0]).mul(100).round(2)[:60]

id                          0.00
referrer                    0.25
src_domain                 10.36
src_url                    10.36
campaign_last               4.16
campaign_first              2.93
at                         89.05
ai                         52.43
same_campaign               0.00
campaign_visits             0.00
campaign_visits_pre         0.00
kw                         85.62
campaign_platform_last      5.18
campaign_platform_first     3.91
tpm                         0.00
cpc                         0.00
dm                          0.00
fbclid                     99.11
gclid                      34.52
msclkid                    80.99
lp_last                     1.37
lp_last_time                0.00
lp_last_dow                 0.00
lp_last_hr                  0.00
lp_first                    0.22
lp_first_time               0.00
lp_first_dow                0.00
lp_first_hr                 0.00
lp_time_diff                0.00
same_lp                     0.00
visit     

In [57]:
df.isna().sum().div(df.shape[0]).mul(100).round(2)[61:120]

gpu_name                   8.46
gpu_x                      0.00
gpu_y                      0.00
sec_as                    57.80
sec_av                     9.74
sec_fw                    47.63
bios_releasedate          10.30
dsk_name                   8.56
dsk_size                   0.00
dsk_iosec                  0.00
dsk_mbsec                  0.00
inst_browser              99.94
web_vistype_browser        6.93
web_vistype_browserver     6.93
web_vistype_os            15.69
web_vistype_gpu           16.64
web_vistype_x             15.69
web_vistype_y             15.69
web_tzo                   22.95
usage                     82.34
usage_ml                  82.33
clientid                   5.12
paid                       0.00
paid_count                 0.00
ltv                        0.00
fa_offers                  0.00
payment_last_time          0.00
payment_last_dow           0.00
payment_last_hr            0.00
payment_last_amount        0.00
payment_first_time         0.00
payment_

In [58]:
df.isna().sum().div(df.shape[0]).mul(100).round(2)[121:148]

mail_spam            0.00
fa_install           0.00
fa_installed         0.00
fa_version_last     23.23
fa_version_first    23.29
fa_uninstalled       0.00
inst_apps            0.00
pc_processes         0.00
pc_software          0.00
pc_hardware          0.00
fa_config            0.00
fa_strategy          0.00
fa_profile           0.00
fa_menustore        41.31
guid                 0.00
fcid                 0.87
event_last_time      0.00
user_agent          12.38
ip                   0.01
geo                  0.00
ip_city              1.35
ip_state_iso         1.33
ip_state_name        1.32
ip_tz                0.38
ip_tzo               0.00
risk                 0.00
d                    0.00
dtype: float64

In [66]:
transac.isna().sum().div(transac.shape[0]).mul(100).round(2)

id                  0.00
customer_id         0.00
subscription_id     0.00
guid                0.61
email               1.89
geo                 0.51
amount              0.00
returns             0.00
item_name           0.26
item_key           12.31
campaign           16.53
state               0.05
created_time        0.00
provider            0.01
oid                57.39
descriptor         28.87
strategy_id        55.69
strategy_config     0.00
json                0.00
dtype: float64

In [65]:
ccsubs.isna().sum().div(ccsubs.shape[0]).mul(100).round(2)

id                    0.00
customer_id           0.00
guid                  0.00
page                  6.46
shopper_id           20.43
created_time          0.00
last_billing_time     0.00
next_billing_time     0.00
amount                0.00
ltv                   0.00
interval_sec          0.00
state                 0.00
retry_count           0.00
descriptor            9.75
meta_data             0.00
log_json              0.00
strategy_id          39.21
strategy_config       0.00
dtype: float64

#### Для набору даних df відкидаю всі колонки з типом object, оскільки замінити на float чи int це занадто проблематично

In [90]:
df_new=df.select_dtypes(exclude=['object'])
df_new

,id,same_campaign,campaign_visits,campaign_visits_pre,cpc,lp_last_time,lp_last_dow,lp_last_hr,lp_first_time,lp_first_dow,...,mail_bounce,mail_blocked,mail_unsub,mail_spam,fa_installed,fa_uninstalled,fcid,event_last_time,ip_tzo,d
0,4568921,0,0,0,0.0000,1584172271,6,3,1581298412,7,...,0,0,0,0,0,0,1.581298e+15,1584448843,240,200210
1,4569179,0,0,0,0.0000,1615308908,2,16,1581336107,1,...,0,0,0,0,0,0,1.581336e+15,1615310358,-60,200210
2,4569217,0,0,0,0.0000,1581767710,6,11,1581342171,1,...,0,0,1,0,0,0,1.581342e+15,1581770092,-60,200210
3,4337476,0,0,0,0.0000,1581365709,1,12,1581365709,1,...,0,0,0,0,0,0,1.559555e+15,1651383434,420,200210
4,4569451,0,0,0,0.0000,1583096985,7,15,1581367053,1,...,0,0,0,0,0,0,1.581367e+15,1583096986,300,200210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3843,10596991,1,2,2,0.2649,1630981480,2,12,1600384785,5,...,0,0,0,0,1,0,1.600385e+15,1662543735,-600,200917
3844,10597129,1,1,1,0.2649,1656771155,6,10,1600385227,4,...,0,0,1,0,1,0,1.600385e+15,1668167166,240,200917
3845,10597258,1,5,5,0.2649,1628100695,3,13,1600385646,4,...,0,0,1,0,1,0,1.600386e+15,1661527722,300,200917
3846,10597400,1,1,1,0.2649,1639536700,2,21,1600386293,4,...,1,1,1,0,1,1,1.600386e+15,1661527722,540,200917


In [91]:
df_new.isna().sum().div(df_new.shape[0]).mul(100).round(2)[:60]

id                          0.00
same_campaign               0.00
campaign_visits             0.00
campaign_visits_pre         0.00
cpc                         0.00
lp_last_time                0.00
lp_last_dow                 0.00
lp_last_hr                  0.00
lp_first_time               0.00
lp_first_dow                0.00
lp_first_hr                 0.00
lp_time_diff                0.00
same_lp                     0.00
download                    0.00
download_last_time          0.00
download_first_time         0.00
download_dur_time          30.77
install_mini_start          0.00
install_mini_start_time     0.00
install_start               0.00
install_start_time          0.00
same_version                0.00
installed                   0.00
install_count               0.00
cogs                        0.00
os_installdate              8.47
os_processes                0.00
os_mem                      0.00
os_virtmem                  0.00
cpu_maxclock                0.00
cpu_cores 

In [92]:
df_new.isna().sum().div(df_new.shape[0]).mul(100).round(2)[61:85]

uninstalled_last_dow      0.00
uninstalled_last_hr       0.00
uninstalled_first_time    0.00
uninstalled_first_dow     0.00
uninstalled_first_hr      0.00
web_uninstall_reason      0.00
survey_no                 0.00
survey_yes                0.00
cpg                       0.00
mail_sending              0.00
email_last_time           0.00
email_first_time          0.00
mail_sent                 0.00
mail_open                 0.00
mail_bounce               0.00
mail_blocked              0.00
mail_unsub                0.00
mail_spam                 0.00
fa_installed              0.00
fa_uninstalled            0.00
fcid                      0.87
event_last_time           0.00
ip_tzo                    0.00
d                         0.00
dtype: float64

#### В таблиці "df_new", для колонок  з пропущеними значеннями менше 10%, замінюю їх  на найчастіше зустрічаюче значення, а для колонок більше 10% - змінюю на наступне значення

In [93]:
df_new['os_installdate'] = df_new['os_installdate'].fillna(df_new['os_installdate'].mode()[0])

In [94]:
df_new['bios_releasedate'] = df_new['bios_releasedate'].fillna(df_new['bios_releasedate'].mode()[0])

In [95]:
df_new['web_vistype_browserver'] = df_new['web_vistype_browserver'].fillna(df_new['web_vistype_browserver'].mode()[0])

In [96]:
df_new['web_vistype_x'] = df_new['web_vistype_x'].fillna(df_new['web_vistype_x'].mode()[0])

In [97]:
df_new['web_vistype_y'] = df_new['web_vistype_y'].fillna(df_new['web_vistype_y'].mode()[0])

In [98]:
df_new['web_tzo'] = df_new['web_tzo'].fillna(df_new['web_tzo'].mode()[0])

In [99]:
df_new['clientid'] = df_new['clientid'].fillna(df_new['clientid'].mode()[0])

In [100]:
df_new['fcid'] = df_new['fcid'].fillna(df_new['fcid'].mode()[0])

In [103]:
df_new.fillna(method='ffill', inplace=True)

In [104]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21425 entries, 0 to 3847
Data columns (total 85 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       21425 non-null  int64  
 1   same_campaign            21425 non-null  int64  
 2   campaign_visits          21425 non-null  int64  
 3   campaign_visits_pre      21425 non-null  int64  
 4   cpc                      21425 non-null  float64
 5   lp_last_time             21425 non-null  int64  
 6   lp_last_dow              21425 non-null  int64  
 7   lp_last_hr               21425 non-null  int64  
 8   lp_first_time            21425 non-null  int64  
 9   lp_first_dow             21425 non-null  int64  
 10  lp_first_hr              21425 non-null  int64  
 11  lp_time_diff             21425 non-null  int64  
 12  same_lp                  21425 non-null  int64  
 13  download                 21425 non-null  int64  
 14  download_last_time     

#### Для таблиці "ccsubs" також для колонок з пропущеними значеннями менше 10%, замінюю їх на найчастіше зустрічаюче значення, а для колонок більше 10% - змінюю на наступне значення, стовпець "strategy_id" видаляю, оскільки велика кількість пропущених значень

In [106]:
ccsubs['page'] = ccsubs['page'].fillna(ccsubs['page'].mode()[0])

In [107]:
ccsubs['descriptor'] = ccsubs['descriptor'].fillna(ccsubs['descriptor'].mode()[0])

In [108]:
ccsubs.fillna(method='ffill', inplace=True)

In [112]:
ccsubs.drop(columns=['strategy_id'], inplace=True)

In [114]:
ccsubs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60932 entries, 0 to 60931
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 60932 non-null  int64  
 1   customer_id        60932 non-null  int64  
 2   guid               60932 non-null  object 
 3   page               60932 non-null  object 
 4   shopper_id         60932 non-null  float64
 5   created_time       60932 non-null  int64  
 6   last_billing_time  60932 non-null  int64  
 7   next_billing_time  60932 non-null  int64  
 8   amount             60932 non-null  float64
 9   ltv                60932 non-null  float64
 10  interval_sec       60932 non-null  int64  
 11  state              60932 non-null  object 
 12  retry_count        60932 non-null  int64  
 13  descriptor         60932 non-null  object 
 14  meta_data          60932 non-null  object 
 15  log_json           60932 non-null  object 
 16  strategy_config    609

#### Для таблиці "transac" також для колонок з пропущеними значеннями менше 10%, замінюю їх на найчастіше зустрічаюче значення, а для колонок більше 10% - змінюю на наступне значення, стовпці "oid", "descriptor", "strategy_id" видаляю, оскільки велика кількість пропущених значень

In [116]:
transac['guid'] = transac['guid'].fillna(transac['guid'].mode()[0])

In [117]:
transac['email'] = transac['email'].fillna(transac['email'].mode()[0])

In [118]:
transac['geo'] = transac['geo'].fillna(transac['geo'].mode()[0])

In [119]:
transac['item_name'] = transac['item_name'].fillna(transac['item_name'].mode()[0])

In [120]:
transac['state'] = transac['state'].fillna(transac['state'].mode()[0])

In [121]:
transac['provider'] = transac['provider'].fillna(transac['provider'].mode()[0])

In [124]:
transac.drop(columns=['oid', 'descriptor', 'strategy_id'], inplace=True)

In [126]:
transac.fillna(method='ffill', inplace=True)

In [128]:
# У стовпці "item_key" залишилось 6,72% порожніх значень, тому замінюї їх на моду
transac['item_key'] = transac['item_key'].fillna(transac['item_key'].mode()[0])

In [130]:
transac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92167 entries, 0 to 92166
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               92167 non-null  object 
 1   customer_id      92167 non-null  object 
 2   subscription_id  92167 non-null  object 
 3   guid             92167 non-null  object 
 4   email            92167 non-null  object 
 5   geo              92167 non-null  object 
 6   amount           92167 non-null  object 
 7   returns          92167 non-null  object 
 8   item_name        92167 non-null  object 
 9   item_key         92167 non-null  object 
 10  campaign         92167 non-null  object 
 11  state            92167 non-null  object 
 12  created_time     92167 non-null  float64
 13  provider         92167 non-null  object 
 14  strategy_config  92167 non-null  object 
 15  json             92167 non-null  object 
dtypes: float64(1), object(15)
memory usage: 11.3+ MB


### Виведення основних показників всіх колонок

In [143]:
df_new.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
id,21425.0,1.497052e+07,9.582898e+06,1.600000e+01,1.016840e+07,1.126562e+07,2.688457e+07,2.927966e+07
same_campaign,21425.0,8.558226e-01,3.512777e-01,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
campaign_visits,21425.0,2.362474e+00,3.744690e+01,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,4.289000e+03
campaign_visits_pre,21425.0,1.456103e+00,8.763784e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,6.660000e+02
cpc,21425.0,6.865680e-01,7.795337e+00,0.000000e+00,2.060000e-01,4.000000e-01,6.155000e-01,9.993000e+02
...,...,...,...,...,...,...,...,...
fa_uninstalled,21425.0,1.421237e-01,3.491851e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
fcid,21425.0,1.610331e+15,4.542279e+13,3.155116e+14,1.591633e+15,1.612714e+15,1.653397e+15,1.668037e+15
event_last_time,21425.0,1.647217e+09,3.468556e+07,1.491145e+09,1.643350e+09,1.664472e+09,1.667945e+09,1.668183e+09
ip_tzo,21425.0,1.314259e+01,3.423217e+03,-3.420600e+05,-1.200000e+02,2.400000e+02,3.000000e+02,8.268000e+04


In [144]:
transac.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,92167,92167,11,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
customer_id,92167,26043,0,320,NaN,NaN,NaN,NaN,NaN,NaN,NaN
subscription_id,92167,41754,0,14488,NaN,NaN,NaN,NaN,NaN,NaN,NaN
guid,92167,23137,E639E267-6FD5-E711-9CC8-80CE62114BE8,742,NaN,NaN,NaN,NaN,NaN,NaN,NaN
email,92167,2169,email@gmail.com,35123,NaN,NaN,NaN,NaN,NaN,NaN,NaN
geo,92167,118,US,38637,NaN,NaN,NaN,NaN,NaN,NaN,NaN
amount,92167.0,274.0,9.99,24840.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
returns,92167.0,246.0,0.0,56043.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
item_name,92167,239,Fast! - 1 PC monthly,30934,NaN,NaN,NaN,NaN,NaN,NaN,NaN
item_key,92167,69,fast_1pc_1m,39059,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
ccsubs.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,60932.0,NaN,NaN,NaN,32372.937619,18903.396305,1.0,15746.75,32372.5,48917.25,64889.0
customer_id,60932.0,NaN,NaN,NaN,191533.517643,61033.668032,1.0,110429.0,212742.0,242539.25,256810.0
guid,60932,22483,E639E267-6FD5-E711-9CC8-80CE62114BE8,174,NaN,NaN,NaN,NaN,NaN,NaN,NaN
page,60932,592,offer#718,8373,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shopper_id,60932.0,NaN,NaN,NaN,253183860.868837,83858522.062014,25938163.0,190094966.0,259041696.0,322852338.0,384245412.0
created_time,60932.0,NaN,NaN,NaN,1637082765.657471,24164046.751176,1496891667.0,1617291366.0,1643642192.0,1658640061.0,1668092494.0
last_billing_time,60932.0,NaN,NaN,NaN,1500146558.952406,457162300.288986,0.0,1612036587.0,1643312284.5,1660229030.25,1668092497.0
next_billing_time,60932.0,NaN,NaN,NaN,532035544.689982,769106309.152481,0.0,0.0,0.0,1624360961.0,1703692793.0
amount,60932.0,NaN,NaN,NaN,66.406031,70.357638,0.0,9.99,39.99,79.99,439.99
ltv,60932.0,NaN,NaN,NaN,45.261888,62.357372,0.0,0.0,24.99,74.99,749.95


#### Перетворення категоріальних даних стовпця "state" таблиці "ccsubs" у цілі числа

In [146]:
unique_values = ccsubs['state'].unique()
unique_values

array(['declined', 'canceled', 'failed', 'recurring', 'once',
       'processing', 'free'], dtype=object)

In [147]:
ccsubs['state_cod'] = ccsubs['state'].map({"declined":1, "canceled":2,
                                                   "failed":3,  "recurring":4, "once":5,
                                                   "processing":6, "free":7})

#### Видалення перетвореного стовпця "state"

In [151]:
ccsubs.drop(columns=['state'], inplace=True)

#### Перейменування створеного стовпця

In [152]:
ccsubs.columns = ccsubs.columns.map(lambda x: x.replace("state_cod", "state"))

In [154]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21425 entries, 0 to 3847
Data columns (total 85 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       21425 non-null  int64  
 1   same_campaign            21425 non-null  int64  
 2   campaign_visits          21425 non-null  int64  
 3   campaign_visits_pre      21425 non-null  int64  
 4   cpc                      21425 non-null  float64
 5   lp_last_time             21425 non-null  int64  
 6   lp_last_dow              21425 non-null  int64  
 7   lp_last_hr               21425 non-null  int64  
 8   lp_first_time            21425 non-null  int64  
 9   lp_first_dow             21425 non-null  int64  
 10  lp_first_hr              21425 non-null  int64  
 11  lp_time_diff             21425 non-null  int64  
 12  same_lp                  21425 non-null  int64  
 13  download                 21425 non-null  int64  
 14  download_last_time     

In [155]:
transac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92167 entries, 0 to 92166
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               92167 non-null  object 
 1   customer_id      92167 non-null  object 
 2   subscription_id  92167 non-null  object 
 3   guid             92167 non-null  object 
 4   email            92167 non-null  object 
 5   geo              92167 non-null  object 
 6   amount           92167 non-null  object 
 7   returns          92167 non-null  object 
 8   item_name        92167 non-null  object 
 9   item_key         92167 non-null  object 
 10  campaign         92167 non-null  object 
 11  state            92167 non-null  object 
 12  created_time     92167 non-null  float64
 13  provider         92167 non-null  object 
 14  strategy_config  92167 non-null  object 
 15  json             92167 non-null  object 
dtypes: float64(1), object(15)
memory usage: 11.3+ MB


In [156]:
ccsubs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60932 entries, 0 to 60931
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 60932 non-null  int64  
 1   customer_id        60932 non-null  int64  
 2   guid               60932 non-null  object 
 3   page               60932 non-null  object 
 4   shopper_id         60932 non-null  float64
 5   created_time       60932 non-null  int64  
 6   last_billing_time  60932 non-null  int64  
 7   next_billing_time  60932 non-null  int64  
 8   amount             60932 non-null  float64
 9   ltv                60932 non-null  float64
 10  interval_sec       60932 non-null  int64  
 11  retry_count        60932 non-null  int64  
 12  descriptor         60932 non-null  object 
 13  meta_data          60932 non-null  object 
 14  log_json           60932 non-null  object 
 15  strategy_config    60932 non-null  object 
 16  state              609

In [ ]:
# Що робити з таблицею transac, де всі колонки з типом object, за виключенням однієї колонки?

In [ ]:
# Як правильно звести дані в одну таблицю, де кожна стрічка буде однією транзакцією - таким чином,
                                            # щоб на кожну транзакцію можна зробити передбачення?
